<a href="https://colab.research.google.com/github/diego2017003/knn_treeClassifier/blob/main/comites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
dados_preProcessado = pd.read_csv('https://raw.githubusercontent.com/diego2017003/knn_treeClassifier/main/dados_preProcessados.csv')
dados_originais = pd.read_csv('https://raw.githubusercontent.com/diego2017003/knn_treeClassifier/main/heart.csv')
dados_preProcessado.drop(columns=['Unnamed: 0'],inplace=True)
dados_originais.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
dados_preProcessado.head()

,age,sex,trtbps,chol,fbs,thalachh,exng,oldpeak,slp_0,slp_1,slp_2,thall_0,thall_1,thall_2,thall_3,caa_0,caa_1,caa_2,caa_3,caa_4,restecg_Normal,restecg_Possui_anormalidade,restecg_Provável_hipertrofia,cp_Angina_atipica,cp_Angina_tipica,cp_Assintomatico,cp_Não_Angina,restecg_Possui_anormalidades,output
0,0.952197,1,0.763956,-0.256334,1,0.015443,0,1.087338,1,0,0,0,1,0,0,1,0,0,0,0,1,-1.003306,0,0,0,0,1,0,1
1,-1.915313,1,-0.092738,0.072199,0,1.633471,0,2.122573,1,0,0,0,0,1,0,1,0,0,0,0,0,0.996705,0,1,0,0,0,1,1
2,-1.474158,0,-0.092738,-0.816773,0,0.977514,0,0.310912,0,0,1,0,0,1,0,1,0,0,0,0,1,-1.003306,0,0,1,0,0,0,1
3,0.180175,1,-0.663867,-0.198357,0,1.239897,0,-0.206705,0,0,1,0,0,1,0,1,0,0,0,0,0,0.996705,0,0,1,0,0,1,1
4,0.290464,0,-0.663867,2.082050,0,0.583939,1,-0.379244,0,0,1,0,0,1,0,1,0,0,0,0,0,0.996705,0,0,0,1,0,1,1


Este notebook terá foco na utilização de comitês para a base de dado que vem sendo trabalhada durante o projeto.     

##1. **Boost**
---

### 1.1 MLPClassifier 

#### Dados originais

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
import numpy as np

class customMLPClassifer(MLPClassifier):
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled


    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and
                                            hasattr(self, "classes_")))

 
X = dados_originais.drop(columns=['output'])
y = dados_originais.output
mlpBoost_10 = AdaBoostClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7,algorithm='SAMME')
mlpBoost_15 = AdaBoostClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7,algorithm='SAMME')
mlpBoost_20 = AdaBoostClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7,algorithm='SAMME')


In [ ]:
'''conda update anaconda-navigator
anaconda-navigator --reset
conda update anaconda-client
conda update -f anaconda-client'''

'conda update anaconda-navigator\nanaconda-navigator --reset\nconda update anaconda-client\nconda update -f anaconda-client'

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
import math
#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=mlpBoost_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=mlpBoost_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=mlpBoost_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8355161814449741 std: 0.06163471188346673
n = 15 Precisão média :0.8108355108084211 std: 0.058694985745623186
n = 20 Precisão média :0.7987703350570998 std: 0.03109644858455377


#### Dados processados

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
mlpBoost_p_10 = AdaBoostClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=10, random_state=7,algorithm='SAMME')
mlpBoost_p_15 = AdaBoostClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=15, random_state=7,algorithm='SAMME')
mlpBoost_p_20 = AdaBoostClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=20, random_state=7,algorithm='SAMME')
#mlpBoost_p_10 = AdaBoostClassifier(base_estimator = customMLPClassifer(activation='identity', hidden_layer_sizes=(12,), solver='adam'),n_estimators=10, random_state=7,algorithm='SAMME')
#mlpBoost_p_15 = AdaBoostClassifier(base_estimator = customMLPClassifer(activation='identity', hidden_layer_sizes=(12,), solver='adam'),n_estimators=15, random_state=7,algorithm='SAMME')
#mlpBoost_p_20 = AdaBoostClassifier(base_estimator = customMLPClassifer(activation='identity', hidden_layer_sizes=(12,), solver='adam'),n_estimators=20, random_state=7,algorithm='SAMME')

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
'''param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,),(22,),(23,),(24,),
             (25,),(26,),(27,),(28,),(29,),(30,),(31,),(32,),(33,),(34,),(35,)
             ]
        }
       ]
clf = GridSearchCV(MLPClassifier(), param_grid, cv=3,scoring='accuracy')

clf.fit(X,y)


print("Best parameters set found on development set:")
print(clf.best_params_)'''

'param_grid = [\n        {\n            \'activation\' : [\'identity\', \'logistic\', \'tanh\', \'relu\'],\n            \'solver\' : [\'lbfgs\', \'sgd\', \'adam\'],\n            \'hidden_layer_sizes\': [\n             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,),(22,),(23,),(24,),\n             (25,),(26,),(27,),(28,),(29,),(30,),(31,),(32,),(33,),(34,),(35,)\n             ]\n        }\n       ]\nclf = GridSearchCV(MLPClassifier(), param_grid, cv=3,scoring=\'accuracy\')\n\nclf.fit(X,y)\n\n\nprint("Best parameters set found on development set:")\nprint(clf.best_params_)'

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=mlpBoost_p_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=mlpBoost_p_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=mlpBoost_p_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))
#n = 10 Precisão média :0.852618703965453 std: 0.05008061533940598
#n = 15 Precisão média :0.8335730682286411 std: 0.06789153517380564
#n = 20 Precisão média :0.8373666868906808 std: 0.06344416567018743

n = 10 Precisão média :0.8568543307397796 std: 0.05216889018898695
n = 15 Precisão média :0.8360385692738633 std: 0.0625089464661355
n = 20 Precisão média :0.8498183421325836 std: 0.05456878192184235


### 1.2 Naive Bayes
---

#### Dados originais

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
import numpy as np


X = dados_originais.drop(columns=['output'])
y = dados_originais.output
nbBoost_10 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=10, random_state=7,algorithm='SAMME')
nbBoost_15 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=15, random_state=7,algorithm='SAMME')
nbBoost_20 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=20, random_state=7,algorithm='SAMME')

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
import math
#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=nbBoost_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=nbBoost_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=nbBoost_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8293866872291795 std: 0.07032541790351075
n = 15 Precisão média :0.839855330428086 std: 0.07196711973407324
n = 20 Precisão média :0.839855330428086 std: 0.07196711973407324


#### Dados processados 

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
nbBoost_p_10 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=10, random_state=7,algorithm='SAMME')
nbBoost_p_15 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=15, random_state=7,algorithm='SAMME')
nbBoost_p_20 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=20, random_state=7,algorithm='SAMME')

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=nbBoost_p_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=nbBoost_p_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=nbBoost_p_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))

n = 10 Precisão média :0.8040560408859474 std: 0.09210672977281531
n = 15 Precisão média :0.8040560408859474 std: 0.09210672977281531
n = 20 Precisão média :0.8040560408859474 std: 0.09210672977281531


### 1.3 Decision Tree
---

#### Dados originais

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [ ]:
X = dados_originais.drop(columns=['output'])
y = dados_originais.output

encoder = LabelEncoder()
binary_encoded_y = pd.Series(encoder.fit_transform(y))

train_X, test_X, train_y, test_y = train_test_split(X, binary_encoded_y, random_state=1)

classifier10 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)
classifier15 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)
classifier20 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)

classifier10.fit(train_X, train_y)
classifier15.fit(train_X, train_y)
classifier20.fit(train_X, train_y)

predictions10 = classifier10.predict(test_X)
predictions15 = classifier15.predict(test_X)
predictions20 = classifier15.predict(test_X)


#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}

scores_10 = cross_validate(estimator=classifier10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))

scores_15 = cross_validate(estimator=classifier15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))

scores_20 = cross_validate(estimator=classifier20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.7917801412983303 std: 0.07335578761678611
n = 15 Precisão média :0.7850042063199958 std: 0.11122956549250076
n = 20 Precisão média :0.7689203065596255 std: 0.10340609908665149


#### Dados processados 

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output

encoder = LabelEncoder()
binary_encoded_y = pd.Series(encoder.fit_transform(y_p))

train_X_p, test_X_p, train_y_p, test_y_p = train_test_split(X_p, binary_encoded_y, random_state=1)

classifier10 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)
classifier15 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)
classifier20 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)

classifier10.fit(train_X_p, train_y_p)
classifier15.fit(train_X_p, train_y_p)
classifier20.fit(train_X_p, train_y_p)

predictions10 = classifier10.predict(test_X_p)
predictions15 = classifier15.predict(test_X_p)
predictions20 = classifier15.predict(test_X_p)

#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}

scores_10 = cross_validate(estimator=classifier10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))

scores_15 = cross_validate(estimator=classifier15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))

scores_20 = cross_validate(estimator=classifier20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8106562305943111 std: 0.08703611819224577
n = 15 Precisão média :0.7908023312686626 std: 0.08528118554121152
n = 20 Precisão média :0.8038512263879911 std: 0.08725376319625677


### 1.4 k-NN

#### Dados originais

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [ ]:
x = dados_originais.drop(columns=['output'])
y = dados_originais.output


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
    
model = KNeighborsClassifier(n_neighbors = 2)
model.fit(X_train, y_train)

predictions10 = model.predict(X_test)
predictions15 = model.predict(X_test)
predictions20 = model.predict(X_test)

#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}

scores_10 = cross_validate(estimator=classifier10, X= x, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))

scores_15 = cross_validate(estimator=classifier15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))

scores_20 = cross_validate(estimator=classifier20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.7999953956861853 std: 0.07228830569260022
n = 15 Precisão média :0.8088064966509239 std: 0.08066862623301459
n = 20 Precisão média :0.7822477494555667 std: 0.0787515050491408


#### Dados processados 

In [ ]:
x_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(x_p, y_p, test_size = 0.2, random_state = 0)
    
model = KNeighborsClassifier(n_neighbors = 2)
model.fit(X_train_p, y_train_p)

predictions10 = model.predict(X_test_p)
predictions15 = model.predict(X_test_p)
predictions20 = model.predict(X_test_p)

confusion_matrix(y_test_p, predictions10)
confusion_matrix(y_test_p, predictions15)

#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}

scores_10 = cross_validate(estimator=classifier10, X= x_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))

scores_15 = cross_validate(estimator=classifier15, X= x_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))

scores_20 = cross_validate(estimator=classifier20, X= x_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.7889824840192488 std: 0.09855290171487256
n = 15 Precisão média :0.7936810274348974 std: 0.0674142294666792
n = 20 Precisão média :0.7895501385439466 std: 0.08130397950576385


## 2. Bagging
---

### 2.1 MLP 

#### Dados originais

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
import numpy as np


X = dados_originais.drop(columns=['output'])
y = dados_originais.output
mlpBag_10 = BaggingClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7)
mlpBag_15 = BaggingClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7)
mlpBag_20 = BaggingClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7)


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
import math
#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=mlpBag_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=mlpBag_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=mlpBag_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8259220640470641 std: 0.08074254261192916
n = 15 Precisão média :0.8416399123752065 std: 0.07116039243816275
n = 20 Precisão média :0.8321176127793775 std: 0.06728938895398007


#### Dados PreProcessados

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
mlpBag_p_10 = BaggingClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=10, random_state=7)
mlpBag_p_15 = BaggingClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=15, random_state=7)
mlpBag_p_20 = BaggingClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=20, random_state=7)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=mlpBag_p_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=mlpBag_p_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=mlpBag_p_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))

n = 10 Precisão média :0.8524393694111186 std: 0.05964772676701073
n = 15 Precisão média :0.8437595782700271 std: 0.049509367139697884
n = 20 Precisão média :0.8473258301296227 std: 0.06359328285413736


### 2.2 Naive Bayes

#### Dados originais

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
import numpy as np


X = dados_originais.drop(columns=['output'])
y = dados_originais.output
nbBag_10 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=10, random_state=7)
nbBag_15 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=15, random_state=7)
nbBag_20 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=20, random_state=7)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=nbBag_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=nbBag_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=nbBag_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8314622484069079 std: 0.059105151157382955
n = 15 Precisão média :0.8184218206973626 std: 0.05917025228746715
n = 20 Precisão média :0.8092972099280148 std: 0.05056133001621758


#### Dados PreProcessados

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
nbBag_p_10 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=10, random_state=7)
nbBag_p_15 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=15, random_state=7)
nbBag_p_20 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=20, random_state=7)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=nbBag_p_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=nbBag_p_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=nbBag_p_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))

n = 10 Precisão média :0.8034839180427416 std: 0.07957654234288679
n = 15 Precisão média :0.8098564793050087 std: 0.07728274294416708
n = 20 Precisão média :0.8169193878017408 std: 0.0742722387493331


### 2.3 Decision Tree

#### Dados originais

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

X = dados_originais.drop(columns=['output'])
y = dados_originais.output

encoder = LabelEncoder()
binary_encoded_y = pd.Series(encoder.fit_transform(y))

train_X, test_X, train_y, test_y = train_test_split(X, binary_encoded_y, random_state=1)

classifier10 = BaggingClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)
classifier15 = BaggingClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)
classifier20 = BaggingClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)

classifier10.fit(train_X, train_y)
classifier15.fit(train_X, train_y)
classifier20.fit(train_X, train_y)

predictions10 = classifier10.predict(test_X)
predictions15 = classifier15.predict(test_X)
predictions20 = classifier15.predict(test_X)

confusion_matrix(test_y, predictions10)
confusion_matrix(test_y, predictions15)
confusion_matrix(test_y, predictions20)

#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}

scores_10 = cross_validate(estimator=classifier10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))

scores_15 = cross_validate(estimator=classifier15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))

scores_20 = cross_validate(estimator=classifier20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8153464440616144 std: 0.07448688410309973
n = 15 Precisão média :0.8203137643230523 std: 0.06927017969684003
n = 20 Precisão média :0.8249288347101814 std: 0.07042931808717201


#### Dados processados 

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output

encoder = LabelEncoder()
binary_encoded_y = pd.Series(encoder.fit_transform(y_p))

train_X_p, test_X_p, train_y_p, test_y_p = train_test_split(X_p, binary_encoded_y, random_state=1)

classifier10 = BaggingClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)
classifier15 = BaggingClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)
classifier20 = BaggingClassifier(DecisionTreeClassifier(max_depth=5),n_estimators=200)

classifier10.fit(train_X_p, train_y_p)
classifier15.fit(train_X_p, train_y_p)
classifier20.fit(train_X_p, train_y_p)

predictions10 = classifier10.predict(test_X_p)
predictions15 = classifier15.predict(test_X_p)
predictions20 = classifier15.predict(test_X_p)

confusion_matrix(test_y_p, predictions10)
confusion_matrix(test_y_p, predictions15)
confusion_matrix(test_y_p, predictions20)

#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}

scores_10 = cross_validate(estimator=classifier10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))

scores_15 = cross_validate(estimator=classifier15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))

scores_20 = cross_validate(estimator=classifier20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8368858446335228 std: 0.08403824389374737
n = 15 Precisão média :0.833853547982805 std: 0.08850199641914654
n = 20 Precisão média :0.8401855783105783 std: 0.07641578342530451


### 2.4 k-NN

#### Dados originais

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

In [ ]:
X = dados_originais.drop(columns=['output'])
y = dados_originais.output

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=0)

classifier10=BaggingClassifier(KNeighborsClassifier(n_neighbors=2),max_samples=0.7,max_features=0.7,n_estimators=1000)
classifier15=BaggingClassifier(KNeighborsClassifier(n_neighbors=2),max_samples=0.7,max_features=0.7,n_estimators=1000)
classifier20=BaggingClassifier(KNeighborsClassifier(n_neighbors=2),max_samples=0.7,max_features=0.7,n_estimators=1000)

classifier10.fit(X_train,y_train)
y_pred=classifier10.predict(X_test)

classifier15.fit(X_train,y_train)
y_pred=classifier15.predict(X_test)

classifier20.fit(X_train,y_train)
y_pred=classifier20.predict(X_test)

#print(classification_report(y_test,y_pred))

scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}

scores_10 = cross_validate(estimator=classifier10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))

scores_15 = cross_validate(estimator=classifier15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))

scores_20 = cross_validate(estimator=classifier20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.6603426883573943 std: 0.10259799886911863
n = 15 Precisão média :0.6663629030927792 std: 0.1047177736092292
n = 20 Precisão média :0.6702513132292544 std: 0.09534061122642604


#### Dados processados 

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output

X_train_p,X_test_p,y_train_p,y_test_p=train_test_split(X_p,y_p,test_size=.3,random_state=0)

classifier10=BaggingClassifier(KNeighborsClassifier(n_neighbors=2),max_samples=0.7,max_features=0.7,n_estimators=1000)
classifier15=BaggingClassifier(KNeighborsClassifier(n_neighbors=2),max_samples=0.7,max_features=0.7,n_estimators=1000)
classifier20=BaggingClassifier(KNeighborsClassifier(n_neighbors=2),max_samples=0.7,max_features=0.7,n_estimators=1000)

classifier10.fit(X_train_p,y_train_p)
y_pred=classifier10.predict(X_test_p)

classifier15.fit(X_train_p,y_train_p)
y_pred=classifier15.predict(X_test_p)

classifier20.fit(X_train_p,y_train_p)
y_pred=classifier20.predict(X_test_p)

#print(classification_report(y_test,y_pred))

scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}

scores_10 = cross_validate(estimator=classifier10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))

scores_15 = cross_validate(estimator=classifier15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))

scores_20 = cross_validate(estimator=classifier20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8320560122282259 std: 0.05275596004726042
n = 15 Precisão média :0.8296277825631542 std: 0.05246521957028134
n = 20 Precisão média :0.8320560122282259 std: 0.05275596004726042


## 3. Stacking

### 3.3 Decision Tree

#### Stacking Homogêneo

##### Dados originais

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
base_learners_10 = []
base_learners_15 = []
base_learners_20 = []
for i in range(10):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('dt_'+str(i),DecisionTreeClassifier()))

for i in range(15):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_15.append(('dt_'+str(i), DecisionTreeClassifier()))

for i in range(20):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_20.append(('dt_'+str(i), DecisionTreeClassifier()))

In [ ]:
X = dados_originais.drop(columns=['output'])
y = dados_originais.output
base_learners = [('rf_1', MLPClassifier())]

dtStk_10 = StackingClassifier(estimators = base_learners_10,cv=3)
dtStk_15 = StackingClassifier(estimators = base_learners_15,cv=3)
dtStk_20 = StackingClassifier(estimators = base_learners_20,cv=3)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=dtStk_10, X= X, y= y, scoring=scoring,cv=3, return_train_score=True)
scores_15 = cross_validate(estimator=dtStk_15, X= X, y= y, scoring=scoring,cv=3, return_train_score=True)
scores_20 = cross_validate(estimator=dtStk_20, X= X, y= y, scoring=scoring,cv=3, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.7240338716961913 std: 0.02250370695634993
n = 15 Precisão média :0.7468114596265857 std: 0.034347868907533066
n = 20 Precisão média :0.7458353559937118 std: 0.044284784846855156


##### Dados processados 

In [ ]:
base_learners_10 = []
base_learners_15 = []
base_learners_20 = []
for i in range(10):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('dt_'+str(i),DecisionTreeClassifier()))

for i in range(15):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_15.append(('dt_'+str(i), DecisionTreeClassifier()))

for i in range(20):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_20.append(('dt_'+str(i), DecisionTreeClassifier()))

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
base_learners = [('rf_1', MLPClassifier())]

dtStk_10 = StackingClassifier(estimators = base_learners_10,cv=3)
dtStk_15 = StackingClassifier(estimators = base_learners_15,cv=3)
dtStk_20 = StackingClassifier(estimators = base_learners_20,cv=3)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=dtStk_10, X= X_p, y= y_p, scoring=scoring,cv=3, return_train_score=True)
scores_15 = cross_validate(estimator=dtStk_15, X= X_p, y= y_p, scoring=scoring,cv=3, return_train_score=True)
scores_20 = cross_validate(estimator=dtStk_20, X= X_p, y= y_p, scoring=scoring,cv=3, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.725002945811792 std: 0.02018420690097352
n = 15 Precisão média :0.7149726749082395 std: 0.041799031877144406
n = 20 Precisão média :0.7023287309879814 std: 0.014726917681363567


#### Stacking Heterogêneo 

##### Dados originais

### 3.4 k-NN


#### Stacking Homogêneo

##### Dados originais

In [ ]:
base_learners_10 = []
base_learners_15 = []
base_learners_20 = []
for i in range(10):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('dt_'+str(i),KNeighborsClassifier()))

for i in range(15):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_15.append(('dt_'+str(i), KNeighborsClassifier()))

for i in range(20):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_20.append(('dt_'+str(i), KNeighborsClassifier()))

X = dados_originais.drop(columns=['output'])
y = dados_originais.output
base_learners = [('rf_1', MLPClassifier())]

dtStk_10 = StackingClassifier(estimators = base_learners_10,cv=3)
dtStk_15 = StackingClassifier(estimators = base_learners_15,cv=3)
dtStk_20 = StackingClassifier(estimators = base_learners_20,cv=3)

scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=dtStk_10, X= X, y= y, scoring=scoring,cv=3, return_train_score=True)
scores_15 = cross_validate(estimator=dtStk_15, X= X, y= y, scoring=scoring,cv=3, return_train_score=True)
scores_20 = cross_validate(estimator=dtStk_20, X= X, y= y, scoring=scoring,cv=3, return_train_score=True)

mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.6558077638401882 std: 0.026784044713219584
n = 15 Precisão média :0.6558077638401882 std: 0.026784044713219584
n = 20 Precisão média :0.6558077638401882 std: 0.026784044713219584


##### Dados processados 

In [ ]:
base_learners_10 = []
base_learners_15 = []
base_learners_20 = []
for i in range(10):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('dt_'+str(i),KNeighborsClassifier()))

for i in range(15):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_15.append(('dt_'+str(i), KNeighborsClassifier()))

for i in range(20):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_20.append(('dt_'+str(i), KNeighborsClassifier()))

X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
base_learners = [('rf_1', MLPClassifier())]

dtStk_10 = StackingClassifier(estimators = base_learners_10,cv=3)
dtStk_15 = StackingClassifier(estimators = base_learners_15,cv=3)
dtStk_20 = StackingClassifier(estimators = base_learners_20,cv=3)

scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=dtStk_10, X= X_p, y= y_p, scoring=scoring,cv=3, return_train_score=True)
scores_15 = cross_validate(estimator=dtStk_15, X= X_p, y= y_p, scoring=scoring,cv=3, return_train_score=True)
scores_20 = cross_validate(estimator=dtStk_20, X= X_p, y= y_p, scoring=scoring,cv=3, return_train_score=True)

mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.79637311105745 std: 0.010409387961367153
n = 15 Precisão média :0.79637311105745 std: 0.010409387961367153
n = 20 Precisão média :0.79637311105745 std: 0.010409387961367153
